In [1]:
import argparse
import os
import numpy as np
import dgl
from dgl import DGLGraph
import dgl.function as fn
from dgl.nn.pytorch import GraphConv
import torch
import torch.nn as nn
import torch.nn.functional as F

from dgl.data import register_data_args, load_data
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.nn.functional import binary_cross_entropy_with_logits as BCELoss
from torch.utils.data import DataLoader
from tqdm import tqdm
from collections import Counter
import pickle
import h5py
import random
import gnn_utils
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import glob2
import scanpy.api as sc
import gae_models
import graph_utils
import seaborn as sns
%load_ext autoreload
%autoreload 2

Using backend: pytorch
/opt/conda/lib/python3.7/site-packages/dgl/base.py:45: DGLWarning: Detected an old version of PyTorch. Suggest using torch>=1.5.0 for the best experience.
  return warnings.warn(message, category=category, stacklevel=1)
/opt/conda/lib/python3.7/site-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,


In [2]:
with open(f'data/params.pickle', 'rb') as handle:
    params = pickle.load( handle)
    
random.seed(params.random_seed)
np.random.seed(params.random_seed)
torch.manual_seed(params.random_seed)
torch.cuda.manual_seed(params.random_seed)

device = gnn_utils.get_device()

In [3]:
path = "."
files = glob2.glob(f'{path}/real_data/*.h5')
files = [f[len(f"{path}/real_data/"):-3] for f in files]
print(files)

['Quake_Smart-seq2_Trachea', 'Quake_Smart-seq2_Diaphragm', 'Quake_10x_Spleen', 'Young', '10X_PBMC_select_2100', 'mouse_ES_cell', 'Adam', 'Quake_10x_Bladder', 'Quake_Smart-seq2_Lung', 'worm_neuron_cell_select_2100', 'Quake_10x_Limb_Muscle', 'worm_neuron_cell', 'mouse_bladder_cell', 'mouse_ES_cell_select_2100', 'Romanov', 'Quake_Smart-seq2_Limb_Muscle', 'Muraro', 'mouse_bladder_cell_select_2100', '10X_PBMC']


In [4]:
sczi_files = [x for x in os.listdir('../dataset') if x.startswith(".")== False]
sczi_files

['Quake_10x_Spleen',
 'Young',
 'Adam',
 'Quake_Smart-seq2_Diaphragm',
 'Quake_Smart-seq2_Trachea',
 'Quake_Smart-seq2_Lung',
 'Romanov',
 'Quake_10x_Limb_Muscle',
 'Bach',
 'Muraro',
 'Quake_10x_Bladder',
 'Quake_Smart-seq2_Limb_Muscle']

In [14]:
for dataset in sczi_files:
    print(f">> {dataset}")
    X, Y, gene_names, cell_names = gnn_utils.get_sczi_data("../", dataset)
    cell_names = np.array(cell_names, dtype='S')
    gene_names = np.array(gene_names, dtype='S')
    dim =X.shape[0]
    if X.shape[0] > 20000:
        continue

    with h5py.File(f"{path}/real_data/{dataset}.h5", 'a') as h:
        # Create dataset inside HDF5 file to store images
        h.create_dataset('X', X.shape, data = X)
        h.create_dataset('Y', Y.shape, data = Y)
        h.create_dataset('cell_names', cell_names.shape, data = cell_names, dtype= h5py.string_dtype())
        h.create_dataset('gene_names', gene_names.shape, data = gene_names, dtype= h5py.string_dtype())
    h.close()

>> Quake_10x_Spleen
Quake_10x_Spleen (9552, 23341)
>> Young
Young (5685, 33658)
>> Adam
Adam (3660, 23797)
>> Quake_Smart-seq2_Diaphragm
Quake_Smart-seq2_Diaphragm (870, 23341)
>> Quake_Smart-seq2_Trachea
Quake_Smart-seq2_Trachea (1350, 23341)
>> Quake_Smart-seq2_Lung
Quake_Smart-seq2_Lung (1676, 23341)
>> Romanov
Romanov (2881, 21143)
>> Quake_10x_Limb_Muscle
Quake_10x_Limb_Muscle (3909, 23341)
>> Bach
Bach (23184, 19965)
>> Muraro
Muraro (2122, 19046)
>> Quake_10x_Bladder
Quake_10x_Bladder (2500, 23341)
>> Quake_Smart-seq2_Limb_Muscle
Quake_Smart-seq2_Limb_Muscle (1090, 23341)


In [ ]:
# !ls real_data